In [ ]:
print("hello world")

hello world


In [32]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
spark=SparkSession.builder.appName("pyspark").getOrCreate()

columns=["customer_id", "cust_name", "city", "grade",  "salesman_id"]

data=[
(70001   ,    150.5    ,   "2012-10-05" , 3005    ,     5002),
(70009   ,    270.65   ,   "2012-09-10" , 3001    ,     5005),
(70002   ,    65.26    ,   "2012-10-05" , 3002    ,     5001),
(70004   ,    110.5    ,   "2012-08-17" , 3009    ,     5003),
(70007   ,    948.5    ,   "2012-09-10" , 3005    ,     5002),
(70005   ,    2400.6   ,   "2012-07-27" , 3007    ,     5001),
(70008   ,    5760.0     ,   "2012-09-10" , 3002    ,     5001),
(70010   ,    1983.43  ,   "2012-10-10" , 3004    ,     5006),
(70003   ,    2480.4   ,   "2012-10-10" , 3009    ,     5003),
(70012   ,    250.45   ,   "2012-06-27" , 3008    ,     5002),
(70011   ,    75.29    ,   "2012-08-17" , 3003    ,     5007),
(70013   ,    3045.6   ,   "2012-04-25" , 3002    ,     5001)
]

orders=spark.createDataFrame(data,columns)
orders.show()

columns1=["customer_id"  ,"cust_name" ,"city" ,"grade ","salesman_id"]

data1=[
(3002   ,      "Nick Rimando" , "New York  " , 100     ,    5001),
(3005   ,      "Graham Zusi " , "California" , 200     ,    5002),
(3001   ,      "Brad Guzan  " , "London    " , 100     ,    5005),
(3004   ,      "Fabian Johns" , "Paris     " , 300     ,    5006),
(3007   ,      "Brad Davis  " , "New York  " , 200     ,    5001),
(3009   ,      "Geoff Camero" , "Berlin    " , 100     ,    5003),
(3008   ,      "Julian Green" , "London    " , 300     ,    5002),
(3003   ,      "Jozy Altidor" , "Moncow    " , 200     ,    5007)
]

customers=spark.createDataFrame(data,columns)
customers.show()

# Define the schema
schema = StructType([
    StructField("salesman_id", StringType(), True),
    StructField("name", StringType(), True),
    StructField("city", StringType(), True),
    StructField("commission", DoubleType(), True)
])

# Data
data2 = [
    ("5001", "James Hoog", "New York", 0.15),
    ("5002", "Nail Knite", "Paris", 0.13),
    ("5005", "Pit Alex", "London", 0.11),
    ("5006", "Mc Lyon", "Paris", 0.14),
    ("5003", "Lauson Hen", "San Jose", 0.12),
    ("5007", "Paul Adam", "Rome", 0.13)
]

# Create the DataFrame
salesman = spark.createDataFrame(data2, schema)

# Display the DataFrame
salesman.show()


####Q. Find salesmen who earned the maximum commission.??

# Find the maximum commission using a subquery
max_commission = salesman.select(max("commission").alias("max_commission")).collect()[0][0]


dfi=orders.join(salesman,orders.salesman_id==salesman.salesman_id,"inner")\
.select(orders["customer_id"], "cust_name", orders["city"], salesman["salesman_id"])\
.filter(col("commission")==max_commission)

dfi.show()

###########################################################################
#lets try to solve this using sql joins
###########################################################################

# Register DataFrames as temporary views
orders.createOrReplaceTempView("orders")
salesman.createOrReplaceTempView("salesman")
customers.createOrReplaceTempView("customers")

# Execute the SQL query
result = spark.sql("""
    SELECT o.customer_id, o.cust_name, o.city, o.salesman_id
    FROM orders o
    INNER JOIN salesman s ON o.salesman_id = s.salesman_id
    WHERE s.commission = (SELECT MAX(commission) FROM salesman)
""")

# Display the result
result.show()

+-----------+---------+----------+-----+-----------+
|customer_id|cust_name|      city|grade|salesman_id|
+-----------+---------+----------+-----+-----------+
|      70001|    150.5|2012-10-05| 3005|       5002|
|      70009|   270.65|2012-09-10| 3001|       5005|
|      70002|    65.26|2012-10-05| 3002|       5001|
|      70004|    110.5|2012-08-17| 3009|       5003|
|      70007|    948.5|2012-09-10| 3005|       5002|
|      70005|   2400.6|2012-07-27| 3007|       5001|
|      70008|   5760.0|2012-09-10| 3002|       5001|
|      70010|  1983.43|2012-10-10| 3004|       5006|
|      70003|   2480.4|2012-10-10| 3009|       5003|
|      70012|   250.45|2012-06-27| 3008|       5002|
|      70011|    75.29|2012-08-17| 3003|       5007|
|      70013|   3045.6|2012-04-25| 3002|       5001|
+-----------+---------+----------+-----+-----------+

+-----------+---------+----------+-----+-----------+
|customer_id|cust_name|      city|grade|salesman_id|
+-----------+---------+----------+-----+-----

In [39]:
#Q.Display Salespeople with More Than One Customer

result1 = spark.sql("""
    SELECT s.salesman_id, s.name, COUNT(o.customer_id) AS customer_count
    FROM salesman s
    JOIN orders o ON s.salesman_id = o.salesman_id
    GROUP BY s.salesman_id, s.name
    HAVING COUNT(o.customer_id) > 1
""")

result1.show()

r2=spark.sql(""" SELECT salesman_id,name FROM salesman a WHERE
salesman_id in(SELECT salesman_id FROM customers group by salesman_id having count(*)>1)""")

r2.show()

############SPARK APPROACH###############

salesmen_with_multiple_customers = customers.groupBy("salesman_id") \
    .agg(count("*").alias("customer_count")) \
    .filter(col("customer_count") > 1) \
    .select(col("salesman_id"))

salesmen_with_multiple_customers.show()

# Join with the 'salesman' DataFrame to get the salesperson's name
result3 = salesmen_with_multiple_customers.join(salesman, on="salesman_id", how="inner") \
    .select("salesman_id", "name")

# Display the result
result3.show()






+-----------+----------+--------------+
|salesman_id|      name|customer_count|
+-----------+----------+--------------+
|       5002|Nail Knite|             3|
|       5003|Lauson Hen|             2|
|       5001|James Hoog|             4|
+-----------+----------+--------------+

+-----------+----------+
|salesman_id|      name|
+-----------+----------+
|       5001|James Hoog|
|       5002|Nail Knite|
|       5003|Lauson Hen|
+-----------+----------+

+-----------+
|salesman_id|
+-----------+
|       5001|
|       5003|
|       5002|
+-----------+

+-----------+----------+
|salesman_id|      name|
+-----------+----------+
|       5001|James Hoog|
|       5002|Nail Knite|
|       5003|Lauson Hen|
+-----------+----------+

